<a href="https://colab.research.google.com/github/yinhang945/CUS635/blob/main/CUS635_Assignment02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#download and import all necessary libraries
import nltk
from nltk import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import pandas as pd
import gensim
import pickle
from gensim.models.word2vec import Word2Vec
from sklearn.manifold import TSNE
import warnings
warnings.filterwarnings('ignore')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [3]:
#load the data file provided in assignment description
with open ('/content/pubmed_abstracts_assignment_2.bin', 'rb') as fp:
    abstract_sample = pickle.load(fp)

In [4]:
#Sneak peek of the data
abstract_sample[0]

'Intervirology. Extensively degraded RNA was isolated from virions of influenza virus which had been oxidized with sodium m-periodate. Similarly, although to a lesser extent, RNA isolated from periodate-treated ribonucleoprotein of influenza virus was also degraded. In contrast, influenza virus RNA, if first freed from other virion components, was not degraded by periodate oxidation.'

In [5]:
#split the dataset into tokens
tokenized_abs_sample = []
for sent in abstract_sample:
  tok_sents = []
  tokens = nltk.word_tokenize(sent)
  for token in tokens:
    tok_sents.append(token)
  tokenized_abs_sample.append(tok_sents)
tokenized_abs_sample[0:2]

[['Intervirology',
  '.',
  'Extensively',
  'degraded',
  'RNA',
  'was',
  'isolated',
  'from',
  'virions',
  'of',
  'influenza',
  'virus',
  'which',
  'had',
  'been',
  'oxidized',
  'with',
  'sodium',
  'm-periodate',
  '.',
  'Similarly',
  ',',
  'although',
  'to',
  'a',
  'lesser',
  'extent',
  ',',
  'RNA',
  'isolated',
  'from',
  'periodate-treated',
  'ribonucleoprotein',
  'of',
  'influenza',
  'virus',
  'was',
  'also',
  'degraded',
  '.',
  'In',
  'contrast',
  ',',
  'influenza',
  'virus',
  'RNA',
  ',',
  'if',
  'first',
  'freed',
  'from',
  'other',
  'virion',
  'components',
  ',',
  'was',
  'not',
  'degraded',
  'by',
  'periodate',
  'oxidation',
  '.'],
 ['Journal',
  'of',
  'general',
  'microbiology',
  '.',
  'Some',
  'mutants',
  'and',
  'stock',
  'strains',
  'of',
  'Escherichia',
  'coli',
  'K12',
  'were',
  'sensitive',
  'to',
  'acriflavine',
  'in',
  'the',
  'presence',
  'of',
  'inorganic',
  'phosphate',
  'but',
  'were

In [6]:
#define pre-processing functions
def remove_punctuation(corpus):
    punctuations = ".,\"-\\/#!?$%\^&\*;:{}=\-_'~()"    
    filtered_corpus = [token for token in corpus if (not token in punctuations)]
    return filtered_corpus

def apply_stopwording(corpus, min_len):
    filtered_corpus = [token.lower() for token in corpus if (not token in stopwords.words('english') and len(token)>min_len)]
    return filtered_corpus

def apply_lemmatization(corpus):
    lemmatizer = nltk.WordNetLemmatizer()
    normalized_corpus = [lemmatizer.lemmatize(token) for token in corpus]
    return normalized_corpus

In [7]:
#apply pre-processing functions to our dataset
g_sentences=[]
for sentence in tokenized_abs_sample:
    g_sentences.append(apply_lemmatization(apply_stopwording(remove_punctuation(sentence),3)))
g_sentences[:2]

[['intervirology',
  'extensively',
  'degraded',
  'isolated',
  'virion',
  'influenza',
  'virus',
  'oxidized',
  'sodium',
  'm-periodate',
  'similarly',
  'although',
  'lesser',
  'extent',
  'isolated',
  'periodate-treated',
  'ribonucleoprotein',
  'influenza',
  'virus',
  'also',
  'degraded',
  'contrast',
  'influenza',
  'virus',
  'first',
  'freed',
  'virion',
  'component',
  'degraded',
  'periodate',
  'oxidation'],
 ['journal',
  'general',
  'microbiology',
  'some',
  'mutant',
  'stock',
  'strain',
  'escherichia',
  'coli',
  'sensitive',
  'acriflavine',
  'presence',
  'inorganic',
  'phosphate',
  'resistant',
  'acriflavine',
  'absence',
  'they',
  'mutated',
  'spontaneously',
  'resistance',
  'acriflavine',
  'plus',
  'phosphate',
  'synergistic',
  'effect',
  'phosphate',
  'acriflavine',
  'sensitivity',
  'increased',
  'high',
  'value',
  'genetic',
  'analysis',
  'suggested',
  'mutation',
  'occurred',
  'gene',
  'acra',
  'electron',
  '

In [8]:
#sneak peek of the after-processing dataset
g_sentences[28:30]

[['clinical',
  'science',
  'molecular',
  'medicine',
  'intracellular',
  'hydrogen',
  'activity',
  'extimated',
  'human',
  'erythrocyte',
  'nucleated',
  'avian',
  'erythrocyte',
  'measurement',
  'distribution',
  'ammonia',
  "5,5'-dimethyloxazolidine-2,4'-dione",
  'intracellular',
  'extracellular',
  'fluid',
  'human',
  'erythrocyte',
  'difference',
  'value',
  'derived',
  'measurement',
  'either',
  'ammonia',
  'avian',
  'erythrocyte',
  'ammonia',
  'consistently',
  'greater',
  'indicating',
  'significant',
  'acid-base',
  'heterogeneity',
  'intracellular',
  'water',
  'degree',
  'heterogeneity',
  'assessed',
  'reference',
  'theoretical',
  'model',
  'compartment',
  'equal',
  'size',
  'experiment',
  'nucleus',
  'isolated',
  'avian',
  'erythrocyte',
  'suggested',
  'bound',
  'nucleoprotein',
  'nucleus',
  'acidic',
  'cytoplasm'],
 ['drug',
  'metabolism',
  'disposition',
  'biological',
  'fate',
  'chemical',
  'metabolism',
  'dihalomet

In [9]:
#generate and save word space embeddings by using embedding size: 1024, windows size: 2, 5, 10 and 20
w2v_model = Word2Vec(sentences=g_sentences, size=1024, sg=1, window = 2, min_count=5, seed = 20, workers=10)
w2v_model2 = Word2Vec(sentences=g_sentences, size=1024, sg=0, window = 2, min_count=5, seed = 20, workers=10)
w2v_model3 = Word2Vec(sentences=g_sentences, size=1024, sg=1, window = 5, min_count=5, seed = 20, workers=10)
w2v_model4 = Word2Vec(sentences=g_sentences, size=1024, sg=0, window = 5, min_count=5, seed = 20, workers=10)
w2v_model5 = Word2Vec(sentences=g_sentences, size=1024, sg=1, window = 10, min_count=5, seed = 20, workers=10)
w2v_model6 = Word2Vec(sentences=g_sentences, size=1024, sg=0, window = 10, min_count=5, seed = 20, workers=10)
w2v_model7 = Word2Vec(sentences=g_sentences, size=1024, sg=1, window = 20, min_count=5, seed = 20, workers=10)
w2v_model8 = Word2Vec(sentences=g_sentences, size=1024, sg=0, window = 20, min_count=5, seed = 20, workers=10)

w2v_model.wv.save_word2vec_format('/content/The_Avengers_pubmed_w2v_uncased_1024_2_skip.emb')
w2v_model2.wv.save_word2vec_format('/content/The_Avengers_pubmed_w2v_uncased_1024_2_cbow.emb')
w2v_model3.wv.save_word2vec_format('/content/The_Avengers_pubmed_w2v_uncased_1024_5_skip.emb')
w2v_model4.wv.save_word2vec_format('/content/The_Avengers_pubmed_w2v_uncased_1024_5_cbow.emb')
w2v_model5.wv.save_word2vec_format('/content/The_Avengers_pubmed_w2v_uncased_1024_10_skip.emb')
w2v_model6.wv.save_word2vec_format('/content/The_Avengers_pubmed_w2v_uncased_1024_10_cbow.emb')
w2v_model7.wv.save_word2vec_format('/content/The_Avengers_pubmed_w2v_uncased_1024_20_skip.emb')
w2v_model8.wv.save_word2vec_format('/content/The_Avengers_pubmed_w2v_uncased_1024_20_cbow.emb')

In [ ]:
w2v_model6.wv.most_similar('greater')

[('higher', 0.7223869562149048),
 ('lower', 0.6596841812133789),
 ('larger', 0.6342519521713257),
 ('stronger', 0.6338860988616943),
 ('smaller', 0.6266578435897827),
 ('greatest', 0.6204589605331421),
 ('equal', 0.5960321426391602),
 ('lesser', 0.5670566558837891),
 ('le', 0.5554319620132446),
 ('fewer', 0.5510126352310181)]

In [ ]:
print(w2v_model8.wv.most_similar(positive=['normal','uncommon'], negative=['common'],topn=5))

[('abnormally', 0.44761747121810913), ('prolactinoma', 0.438173770904541), ('abnormal', 0.4374072551727295), ('neurologically', 0.42681434750556946), ('minora', 0.41136234998703003)]


In [ ]:
print(w2v_model4.wv.most_similar(positive=['husband','female'], negative=['male'],topn=5))

[('wife', 0.7636353969573975), ('married', 0.7080977559089661), ('homosexual', 0.6985017657279968), ('same-sex', 0.6932131052017212), ('marriage', 0.6837477684020996)]


In [ ]:
print(w2v_model2.wv.most_similar(positive=['increase','maximise'], negative=['minimise'],topn=5))

[('decrease', 0.6512426137924194), ('diminution', 0.5949548482894897), ('decline', 0.5458647012710571), ('reduction', 0.5304989814758301), ('decrement', 0.5239008665084839)]


In [ ]:
print(w2v_model6.wv.most_similar(positive=['lower','larger'], negative=['smaller'],topn=5))

[('higher', 0.8294529914855957), ('greater', 0.6044450998306274), ('poorer', 0.5218620300292969), ('stronger', 0.5190677642822266), ('highest', 0.497153639793396)]
